In [2]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import List, Dict, Any
from nuscenes.utils.geometry_utils import BoxVisibility, transform_matrix
from pyquaternion import Quaternion
import pandas as pd
from tqdm import tqdm

import math
from scipy.spatial.transform import Rotation as R

In [4]:
root = '/data/lei/DeepVO/nuScenes_dataset/nuscenes/' # location of database
nusc = NuScenes(version='v1.0-trainval', dataroot=root)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 97.090 seconds.
Reverse indexing ...
Done reverse indexing in 6.9 seconds.


Create map-scenes

In [3]:
map_scene = {}
for i in range(850):
    if nusc.get('log', nusc.scene[i]['log_token'])['location'] not in map_scene.keys():
        # print(True)
        map_scene[nusc.get('log', nusc.scene[i]['log_token'])['location']] = []
    map_scene[nusc.get('log', nusc.scene[i]['log_token'])['location']].append(str(i).zfill(3))

In [4]:
map_scene

{'singapore-onenorth': ['000',
  '001',
  '002',
  '003',
  '004',
  '005',
  '006',
  '007',
  '008',
  '009',
  '010',
  '011',
  '012',
  '013',
  '014',
  '015',
  '016',
  '017',
  '018',
  '019',
  '020',
  '021',
  '022',
  '023',
  '024',
  '025',
  '026',
  '027',
  '028',
  '029',
  '030',
  '031',
  '032',
  '033',
  '034',
  '035',
  '036',
  '037',
  '038',
  '039',
  '040',
  '041',
  '042',
  '043',
  '044',
  '045',
  '046',
  '047',
  '048',
  '049',
  '050',
  '051',
  '052',
  '053',
  '054',
  '055',
  '056',
  '057',
  '058',
  '093',
  '094',
  '095',
  '096',
  '097',
  '098',
  '099',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '147',
  '148',
  '149',
  '150',
  '151',
  '152',
  '153',
  '171',
  '214',
  '215',
  '216',
  '217',
  '218',
  '219',
  '220',
  '221',
  '222',
  '223',
  '224',
  '261',
  '262',
  '263

Generate 12hz version scenes

In [10]:
image_path = './NUSC_12hz/CAM_FRONT/sequences/'

for i in tqdm(range(850)):
    
    if not os.path.exists(image_path + '{:03d}/'.format(i)):
        os.mkdir(image_path + '{:03d}/'.format(i))
        os.mkdir(image_path + '{:03d}/'.format(i) + 'image_2/')
        
    first_sample = nusc.get('sample', nusc.scene[i]['first_sample_token'])
    sample_data = nusc.get('sample_data', first_sample['data']['CAM_FRONT'])
    num = 0
    while True:
        # print(sample_data['token'])
        # if sample_data['is_key_frame']:
        #     print(True)

        # image = Image.open(root + sample_data['filename'])

        # image.save(image_path + '{:03d}/'.format(i) + 'image_2/' + '{:06d}.jpg'.format(num))

        os.system('cp ' + root + sample_data['filename'] + ' ./NUSC_12hz/CAM_FRONT/sequences/{:03d}/image_2/{}.jpg'.format(i, str(num).zfill(6)))

        num = num + 1
                            
        if sample_data['next'] != '':
            sample_data = nusc.get('sample_data', sample_data['next'])
        else:
            break

100%|██████████| 850/850 [31:18<00:00,  2.21s/it]


Generate pose (standard version)

In [6]:
for num in range(850):

    current_scene = nusc.scene[num]
    sample_token = current_scene['first_sample_token']
    sample = nusc.get('sample', sample_token)
    cam_token = sample['data']['CAM_FRONT']
    cam = nusc.get('sample_data', cam_token)

    first_pose = nusc.get('ego_pose', cam['ego_pose_token'])
    first_pose_mat = transform_matrix(first_pose['translation'], Quaternion(first_pose['rotation']), inverse=False)

    first_calibrated_sensor_token = cam['calibrated_sensor_token']
    _first_calibrate_sensor = nusc.get('calibrated_sensor', first_calibrated_sensor_token)
    first_cam_pose_mat = transform_matrix(_first_calibrate_sensor['translation'], Quaternion(_first_calibrate_sensor['rotation']), inverse=False)

    cords = np.zeros((1, 4))
    cords[:,-1] = 1

    x = []
    y = []
    with open('/data/lei/DeepVO/nuScenes_dataset/NUSC_12hz/CAM_FRONT/poses/{}.txt'.format(str(num).zfill(3)), 'w') as f:
        while True:
            cam = nusc.get('sample_data', cam_token)
            calibrated_sensor_token = cam['calibrated_sensor_token']

            pose = nusc.get('ego_pose', cam['ego_pose_token'])
            cur_pose_mat = transform_matrix(pose['translation'], Quaternion(pose['rotation']), inverse=False)

            _calibrate_sensor = nusc.get('calibrated_sensor', calibrated_sensor_token)
            cur_cam_pose_mat = transform_matrix(_calibrate_sensor['translation'], Quaternion(_calibrate_sensor['rotation']), inverse=False)
            
            _temp_pose = np.dot(np.linalg.inv(first_cam_pose_mat), np.linalg.inv(first_pose_mat))
            _new_pose = np.dot(_temp_pose, cur_pose_mat)
            _new_pose = np.dot(_new_pose, cur_cam_pose_mat)

            _rotation = _new_pose[:3, :3]

            # print(R.from_matrix(_rotation).as_euler('XYZ', degrees=True))

            _loc = np.dot(_new_pose, np.transpose(cords))

            x.append(_loc[0][0])
            y.append(_loc[2][0])

            f.write(str(_rotation[0][0]) + ' ')
            f.write(str(_rotation[0][1]) + ' ')
            f.write(str(_rotation[0][2]) + ' ')

            f.write(str(_loc[0][0]) + ' ')

            f.write(str(_rotation[1][0]) + ' ')
            f.write(str(_rotation[1][1]) + ' ')
            f.write(str(_rotation[1][2]) + ' ')

            f.write(str(_loc[1][0]) + ' ')

            f.write(str(_rotation[2][0]) + ' ')
            f.write(str(_rotation[2][1]) + ' ')
            f.write(str(_rotation[2][2]) + ' ')

            f.write(str(_loc[2][0]) + '\n')

            cam_token = cam['next']

            if cam_token == '':
                break
